In [1]:
%matplotlib inline

import pykat
import numpy as np

import pykat.ifo.aligo as aligo
import pykat.ifo.aligo.plot
import matplotlib.pyplot as plt

import scipy
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d

pykat.init_pykat_plotting(fmts=['svg'])

C:\Users\tvo88\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


                                              ..-
    PyKat 1.1.277         _                  '(
                          \`.|\.__...-""""-_." )
       ..+-----.._        /  ' `            .-'
   . '            `:      7/* _/._\    \   (
  (        '::;;+;;:      `-"' =" /,`"" `) /
  L.        \`:::a:f            c_/     n_'
  ..`--...___`.  .    ,
   `^-....____:   +.      www.gwoptics.org/pykat



In [2]:
%run LHO.py

base = make_LHO()
base.verbose = False
base.dumpify()

# We take a pickoff of the OMC refl and measure the beamscan along there
base.parse("s sOMC_REFL 2 nOMC_ICb nOMC_REFL")

# Data taken for matching the OPO beam to the OMC
# Furthest, Middle, and Closest refer to the second adjustable
# lens position after the OPO. Close means closest to the OPO
Furthest = np.array([[-0.2635,-0.2285,-0.2005,-0.1555,-0.1135],
                     [0.000621,0.000647,0.0006675,0.000714,0.000757],
                     [0.0004515,0.00046,0.000475,0.00049,0.00052]])

Closest  = np.array([[-0.2635,-0.2379,-0.1935,-0.1585,-0.1175],
                     [0.000334,0.000326,0.0003175,0.0003355,0.0003745],
                     [0.000358,0.000344,0.000327,0.000332,0.000354   ]])

Middle   = np.array([[-0.2635,-0.2379,-0.1935,-0.1615,-0.1205         ,1.2255,1.2255,1.2972,1.2972,1.3588,1.3588,1.419,1.419,1.474,1.474  ],
                     [0.000395,0.0003825,0.0004105,0.00043,0.0004585  ,0.001835,0.00182,0.00185,0.001885,0.001935,0.001825,0.002,0.002,0.00185,0.00205   ],
                     [0.000374,0.000373,0.0003735,0.0003885,0.0004055 ,0.001315,0.0013,0.00128,0.00136,0.00136,0.00139,0.001415,0.001345,0.00131,0.001365 ]])

OPO      = np.array([[2.5*25.4e-3, 28.5*25.4e-3, 39.75*25.4e-3, (16.5-2.125)*25.4e-3],
                     [2320e-6, 1830e-6, 1680e-6, 2030e-6],
                     [2301e-6, 1730e-6, 1590e-6, 2000e-6]])


OPO_wo_lens2 = np.array([[2.25*25.4e-3, 4.5*25.4e-3],
                         [4454e-6     , 4918e-6    ],
                         [4530e-6     , 4816e-6    ]])

Middle_SRM_refl_HAM6 = np.array([[1.36, 1.33, 1.3081, 0.4826, 0.433, 0.381 ],
                                 [1786e-6, 1787e-6, 1782e-6, 1768e-6, 1776e-6, 1784e-6],
                                 [2332e-6, 2216e-6, 2209e-6, 2017e-6, 2022e-6, 2043e-6]])

def cav2node(cav_name, node, verbose =True, base=base):
    """
    This method traces a cavity's eigen mode to a
    given node in the model. 
    
    cav_name: cavity name
    node: node to trace beam to
    verbose: More or less info
    base: The kat object to do the tracing with
    """
    kat = base.deepcopy()

    for cav in kat.getAll(pykat.commands.cavity):
        if cav_name not in cav.name:
            cav.remove()

    kat.noxaxis = True
    kat.parse("pd p nAS")

    _, T = kat.run(getTraceData=True)
    qx, qy, _ = T[0][node]
    
    if verbose: print("%s -> %s" % (cav_name, node), qx.q, qy.q)
    return qx, qy


def single_bounce_SQZ(base, direction = 'x', verbose=True, to_node='nAS'):
    """
    This does a single bounce trace off of SRM HR surface from the OPO beam.
    This happens in two parts. Once trace from the OPO to the SRM HR
    a refl off of SRM HR, and then a trace from this reflected beam to
    the requested node.
    
    Returns the final q value going in to the OMC
    """
    SQZqx = base.sqz.nSQZ.qx 
    SQZqy = base.sqz.nSQZ.qy
    
    if direction == 'y':
        qin = SQZqy
    else:
        qin = SQZqx

    SQZ2SRM = base.beamTrace(qin, "nSQZ", "nSRMs1", direction=direction)
    
    #ITMX substrate
    n = base.SRMsub.n.value

    # Manually calculate the reflection off of SRM HR inside substrate
    q2 = pykat.optics.ABCD.apply(base.SRM.ABCD("nSRMs1", "nSRMs1", direction=direction),
                                 SQZ2SRM.q_out,
                                 n, n)

    if to_node != "nSRMs1":
        SRM2OMC = base.beamTrace(q2, "nSRMs1", to_node, direction=direction)

        if verbose:
            SQZ2SRM.print()
            SRM2OMC.print()

        return pykat.BeamParam(q=SRM2OMC.q_out)
    else:
        if verbose:
            SQZ2SRM.print()
        
        return pykat.BeamParam(q=SQZ2SRM.q_out)

ERROR:root:File `'LHO.py'` not found.


NameError: name 'make_LHO' is not defined

In [ ]:
def func_sqz_middle_fit(x, dz1, dz2, direction, base):
    kat = base.deepcopy()

    lens2_actuator_pos = 0.5
    
    if direction == 'x':
        qin = kat.sqz.nSQZ.qx
    else:
        qin = kat.sqz.nSQZ.qy
    
    kat.data['fn_OPO_shift_lenses'](kat, dz1, lens2_actuator_pos, dz2)
    
    trace_SQZ_SRM = kat.beamTrace(qin, 'nSQZ', "nSRMs1", direction)
    z, w = trace_SQZ_SRM.beamsize_z(5e-3)
    
    return interp1d(z, w)(x)

# Fit to ZM1 beamscan

We took a beam scan after ZM1 with lens2 in the middle position. Here we fit this beam shape by varying the q value used at the squeezer component. This is then the beam shape that is propagating towards the SRM. Assuming the lengths and lenses before lens2 are good, we can vary the lens2 position and predict the beam at the OMC later.

In [ ]:
kat = base.deepcopy()

def func_sqz_middle_fitx(x, dz1, dz2):
    return func_sqz_middle_fit(x, dz1, dz2, 'x', kat)

def func_sqz_middle_fity(x, dz1, dz2):
    return func_sqz_middle_fit(x, dz1, dz2, 'y', kat)

trace_SQZ_SRM_HR = kat.beamTrace(base.sqz.nSQZ.qx, 'nSQZ', "nSRMs1",'x')
ZM1_pos          = trace_SQZ_SRM_HR.data['ZM1']['z']

xdata = ZM1_pos + OPO[0,:]
ydata = OPO[1,:]/2e-3
p0x = (0, 0)
poptlensx, covx = curve_fit(func_sqz_middle_fitx, xdata, ydata, p0=p0x)

xdata = ZM1_pos + OPO[0,:]
ydata = OPO[2,:]/2e-3
p0y = (0, 0)
poptlensy, covy = curve_fit(func_sqz_middle_fity, xdata, ydata, p0=p0y)

display("x fit [dz1, dz2]", poptlensx, "in", p0x)
display("y fit [dz1, dz2]", poptlensy, "in", p0y)

In [ ]:
z = np.linspace(0, trace_SQZ_SRM_HR.data['nSRMs1']['z']-0.01, 1000)

plt.figure(figsize=(5*1.6, 4))
#plt.plot(z, func_sqz_middle_fitx(z, 0, 0), 'r--', label='x (initial)')
#plt.plot(z, func_sqz_middle_fity(z, 0, 0), 'b--',label='y (initial)')

plt.plot(z, func_sqz_middle_fitx(z, *poptlensx), 'r', label='x')
plt.plot(z, func_sqz_middle_fity(z, *poptlensy),  'b', label='y')

fval = base.sqz_lens2.f.value
kat.sqz_lens2.f = np.inf

plt.plot(z, func_sqz_middle_fitx(z, *poptlensx), 'k--', label='x (wo lens2)')
plt.plot(z, func_sqz_middle_fity(z, *poptlensy), 'k:',label='y (wo lens2)')

kat.sqz_lens2.f = fval

plt.scatter(ZM1_pos + OPO[0,:], OPO[1,:]/2e-3, 50, color='r', marker='x', label='meas (x)')
plt.scatter(ZM1_pos + OPO[0,:], OPO[2,:]/2e-3, 50, color='b', marker='+', label='meas (y)')

plt.scatter(ZM1_pos + OPO_wo_lens2[0,:], OPO_wo_lens2[1,:]/2e-3, 50, color='k', marker='x', label='meas wo lens 2 (x)')
plt.scatter(ZM1_pos + OPO_wo_lens2[0,:], OPO_wo_lens2[2,:]/2e-3, 50, color='k', marker='x', label='meas wo lens 2 (y)')

plt.xlabel("Position from M1 HR [m]")
plt.ylabel("Beamsize [mm]")
plt.legend(ncol=4, loc="upper center")
plt.title("OPO -> SRM Fit")
plt.xlim(0, 2.5)
plt.ylim(0, 3.2)





# Set base model to use lens offsets that fit ZM1 data

In [ ]:
z = np.linspace(0, 3, 1000)
dz1, dz2 = np.mean((poptlensx,), 0)

print("average lens offsets to use:", dz1, dz2)

base.data['fn_OPO_shift_lenses'](base, dz1, 0.5, dz2)

plt.plot(z, func_sqz_middle_fitx(z, dz1, dz2), 'r--', label='x (initial)')
plt.plot(z, func_sqz_middle_fity(z, dz1, dz2), 'b--', label='y (initial)')

plt.scatter(ZM1_pos + OPO[0,:], OPO[1,:]/2e-3, 50, color='r', marker='x', label='meas (x)')
plt.scatter(ZM1_pos + OPO[0,:], OPO[2,:]/2e-3, 50, color='b', marker='x', label='meas (y)')

plt.xlabel("Position from M1 HR [m]")
plt.ylabel("Beamsize [mm]")
plt.legend()
plt.title("Fit")


# Project beams to OMC

In [ ]:
def SRM_HR_substrate_refl(qix, qiy):
    qx = pykat.optics.ABCD.apply(base.SRM.ABCD("nSRMs1", "nSRMs1", direction='x'),
                                     qix, kat.SRMsub.n.value, kat.SRMsub.n.value)
    qy = pykat.optics.ABCD.apply(base.SRM.ABCD("nSRMs1", "nSRMs1", direction='y'),
                                     qiy, kat.SRMsub.n.value, kat.SRMsub.n.value)
    
    return qx, qy

def run(kat):
    """
    This function runs the provided kat object to trace the beam form the
    OPO to the OMC. It plots the data taken against the model generated.
    The parameters of the model can be changed by changing the input kat.
    The popt x/y are the optimised qx and qy from the fitting before of the
    beam coming out of the OPO after ZM1.
    """
    fig = plt.figure(figsize=(4*2, 4))
    ax = plt.subplot(111)

    fig2 = plt.figure(figsize=(4*2, 4))
    ax2 = plt.subplot(111)

    # Get cavity OMC q at SRM...
    OMCqx, OMCqy = cav2node('OMC', 'nSRMs1', kat)
    OMCqx.reverse()
    OMCqy.reverse()
    
    # Then trace it back to get the beam shape
    trace_OMCx = kat.beamTrace(OMCqx, 'nSRMs1', "nOMC_REFL",'x')
    trace_OMCx.plot_beamsize(ax=ax2, color='k', ls='--', label="OMC")
    trace_OMCy = kat.beamTrace(OMCqy, 'nSRMs1', "nOMC_REFL",'y')
    trace_OMCy.plot_beamsize(ax=ax2, color='k', ls=':', label="OMC")
    
    print(OMCqx, OMCqy)
    
    # Loop over the three lens2 options and do the tracing
    for lens2_pos,c,pos in zip([0.05, 0.5, 0.95], ['r','g','b'], ['Far', 'Middle', 'Close']):
        #shift_lenses(kat, 0, lens2_pos)
        kat.data['fn_OPO_shift_lenses'](kat, dz1, lens2_pos, dz2)
        
        trace_SQZ_SRMx = kat.beamTrace(kat.sqz.nSQZ.qx, 'nSQZ', "nSRMs1",'x')
        trace_SQZ_SRMx.plot_beamsize(ax=ax, color=c, label=pos + '(x)')
        trace_SQZ_SRMy = kat.beamTrace(kat.sqz.nSQZ.qy, 'nSQZ', "nSRMs1",'y')
        trace_SQZ_SRMy.plot_beamsize(ax=ax, color=c, label=pos + '(y)', ls='--')
        
        qx, qy = SRM_HR_substrate_refl(trace_SQZ_SRMx.q_out, trace_SQZ_SRMy.q_out)
        
        trace_SRM_OMCx = kat.beamTrace(qx, 'nSRMs1', "nOMC_REFL",'x')
        trace_SRM_OMCx.plot_beamsize(ax=ax2, color=c, label=pos + '(x)')
        trace_SRM_OMCy = kat.beamTrace(qy, 'nSRMs1', "nOMC_REFL",'y')
        trace_SRM_OMCy.plot_beamsize(ax=ax2, color=c, label=pos + '(y)', ls='--')

        print(pos, "x", pykat.BeamParam.overlap(trace_SRM_OMCx.q_out, trace_OMCx.q_out))
        print(pos, "y", pykat.BeamParam.overlap(trace_SRM_OMCy.q_out, trace_OMCy.q_out))

    OMC_waist_pos = trace_OMCx.data['OM3']['z'] + 0.117 + 0.2 + 0.141
    ZM1_pos = trace_SQZ_SRM_HR.data['ZM1']['z']

    # Factor of two because measurement in diameter
    ax.scatter(ZM1_pos + OPO[0,:], OPO[1,:]/2e-3, color='k', marker='x')
    ax.scatter(ZM1_pos + OPO[0,:], OPO[2,:]/2e-3, color='k', marker='+')
    
    ax2.scatter(OMC_waist_pos + Furthest[0,:], Furthest[1,:]/1e-3, color='r', marker='x', label='meas x')
    ax2.scatter(OMC_waist_pos + Middle[0,:],   Middle[1,:]/1e-3, color='g', marker='x',   label='meas x')
    ax2.scatter(OMC_waist_pos + Closest[0,:],  Closest[1,:]/1e-3, color='b', marker='x',  label='meas x')

    ax2.scatter(OMC_waist_pos + Furthest[0,:], Furthest[2,:]/1e-3, color='r', marker='+', label='meas y')
    ax2.scatter(OMC_waist_pos + Middle[0,:],   Middle[2,:]/1e-3, color='g', marker='+',   label='meas y')
    ax2.scatter(OMC_waist_pos + Closest[0,:],  Closest[2,:]/1e-3, color='b', marker='+',  label='meas y')

    OM1_pos = trace_OMCx.data['OM1']['z']
    
    ax2.scatter(OM1_pos - Middle_SRM_refl_HAM6[0,:], Middle_SRM_refl_HAM6[1,:]/2e-3, color='g', marker='x')
    ax2.scatter(OM1_pos - Middle_SRM_refl_HAM6[0,:], Middle_SRM_refl_HAM6[2,:]/2e-3, color='g', marker='+')
    
    Middle_SRM_refl_HAM6
    
    
    ax.set_xlabel("OPO -> SRM")
    ax.legend()
    #ax.set_xlim(1,1.5)
    ax.set_ylim(0,2)

    ax2.set_xlabel("SRM -> OMC REFL")
    ax2.legend(ncol=2)
    ax2.set_ylim(0,3)

    ax2.set_xlim(0, None)
    ax2.set_ylim(0.2, 1.9)

In [ ]:
kat = base.deepcopy()

kat.lzm1_zm2.L *= 1
kat.sSRM_FI.L *= 1
kat.sFI_OM1.L *= 1
kat.sOM1_OM2.L *= 1

kat.SRM.Rcx *= 0
kat.SRM.Rcy *= 0

#kat.sqz_lens1.f = 0.250 *2.23
#kat.sqz_lens2.f = 0.350*2.23
run(kat)


This fit looks reasonable considering we still don't have a good explanation for the amount of astigmatism. However the waist position and size of the closest and middle values are looking comparable to https://alog.ligo-wa.caltech.edu/aLOG/index.php?callRep=41504. Furthest is still a bit odd. The astigmatism is large, perhaps the lens2 is not centered or shifted...

Assuming this represents the close and middle results well though we can now shift the lens even closer and check the mismatching to the OMC.

In [ ]:
def run_change_lens2(kat):
    fig2 = plt.figure(figsize=(4*2, 4))
    ax2 = plt.subplot(111)

    OMCqx, OMCqy = cav2node('OMC', 'nSRM2', kat)
    OMCqx.reverse()
    OMCqy.reverse()

    trace_OMCx = kat.beamTrace(OMCqx, 'nSRM2', "nOMC_REFL",'x')
    trace_OMCx.plot_beamsize(ax=ax2, color='k', ls='--', label="OMC")

    trace_OMCy = kat.beamTrace(OMCqy, 'nSRM2', "nOMC_REFL",'y')
    trace_OMCy.plot_beamsize(ax=ax2, color='k', ls=':', label="OMC")

    # Here shift the lens 2 and 4 cm closer compared to the cloest position, and see what happens...
    for dl,c,pos in zip([0e-2, 2e-2, 4e-2], ['r','g','b'], ['0cm', '2cm', '4cm']):
        pos = "%.2f cm" % (dl*100)
        lens2_pos = 0.95 # factor of 0.95 for total 42mm range, is kind of right for the closest measurement we did before
        
        kat.data['fn_OPO_shift_lenses'](kat, 0, lens2_pos, dl)
        
        trace_SQZ_SRMx = kat.beamTrace(kat.sqz.nSQZ.qx, 'nSQZ', "nSRMs2",'x')
        
        qx = single_bounce_SQZ(kat, 'x', False, 'nSRM2')

        trace_SRM_OMCx = kat.beamTrace(qx, 'nSRM2', "nOMC_REFL",'x')
        trace_SRM_OMCx.plot_beamsize(ax=ax2, color=c, label=pos + '(x)')

        trace_SQZ_SRMy = kat.beamTrace(kat.sqz.nSQZ.qy, 'nSQZ', "nSRMs2",'y')
        
        qy = single_bounce_SQZ(kat, 'y', False, 'nSRMs2')

        trace_SRM_OMCy = kat.beamTrace(qy, 'nSRMs2', "nOMC_REFL",'y')
        trace_SRM_OMCy.plot_beamsize(ax=ax2, color=c, label=pos + '(y)', ls='--')

        print(pos, "x mismatch", pykat.BeamParam.overlap(trace_SRM_OMCx.q_out, trace_OMCx.q_out))
        print(pos, "y mismatch", pykat.BeamParam.overlap(trace_SRM_OMCy.q_out, trace_OMCy.q_out))

    ax2.set_xlabel("SRM AR -> OMC REFL")
    ax2.legend()
    ax2.set_ylim(0,3)

    ax2.set_xlim(5.3, 6.5)
    ax2.set_ylim(0, 1)

In [ ]:
kat = base.deepcopy()
kat.lzm1_zm2.L = 2260e-3
kat.sOM1_OM2.L = 1.39
run_change_lens2(kat)

So we see that the mode matching gets worse as we move lens2 closer...

The waist position gets better but it also gets smaller which hurts us.

In [ ]:
def run_change_lens1(kat):
    fig2 = plt.figure(figsize=(4*2, 4))
    ax2 = plt.subplot(111)

    OMCqx, OMCqy = cav2node('OMC', 'nSRM2', kat)
    OMCqx.reverse()
    OMCqy.reverse()

    trace_OMCx = kat.beamTrace(OMCqx, 'nSRM2', "nOMC_REFL",'x')
    trace_OMCx.plot_beamsize(ax=ax2, color='k', ls='--', label="OMC")

    trace_OMCy = kat.beamTrace(OMCqy, 'nSRM2', "nOMC_REFL",'y')
    trace_OMCy.plot_beamsize(ax=ax2, color='k', ls=':', label="OMC")

    # Here shift the lens 2 and 4 cm closer compared to the cloest position, and see what happens...
    for dl,c,pos in zip([2e-2, 4e-2, 10e-2], ['r','g','b'], ['0cm', '2cm', '4cm']):
        pos = "%.2f cm" % (dl*100)
        
        kat.data['fn_OPO_shift_lenses'](kat, dl, 0.95, 10e-2)
        
        trace_SQZ_SRMx = kat.beamTrace(kat.sqz.nSQZ.qx, 'nSQZ', "nSRMs2",'x')
        
        qx = single_bounce_SQZ(kat, 'x', False, 'nSRM2')

        trace_SRM_OMCx = kat.beamTrace(qx, 'nSRM2', "nOMC_REFL",'x')
        trace_SRM_OMCx.plot_beamsize(ax=ax2, color=c, label=pos + '(x)')

        trace_SQZ_SRMy = kat.beamTrace(kat.sqz.nSQZ.qy, 'nSQZ', "nSRMs2",'y')
        
        qy = single_bounce_SQZ(kat, 'y', False, 'nSRMs2')

        trace_SRM_OMCy = kat.beamTrace(qy, 'nSRMs2', "nOMC_REFL",'y')
        trace_SRM_OMCy.plot_beamsize(ax=ax2, color=c, label=pos + '(y)', ls='--')

        #print(kat.lsqz_lens1.L.value, kat.lsqz_lens1_lens2.L.value, kat.lsqz_lens2_zm1.L.value)
        print(pos, "x mismatch", pykat.BeamParam.overlap(trace_SRM_OMCx.q_out, trace_OMCx.q_out))
        print(pos, "y mismatch", pykat.BeamParam.overlap(trace_SRM_OMCy.q_out, trace_OMCy.q_out))

    ax2.set_xlabel("SRM AR -> OMC REFL")
    ax2.legend()
    ax2.set_ylim(0,3)

    ax2.set_xlim(5.3, 6.5)
    ax2.set_ylim(0, 1)

In [ ]:
kat = base.deepcopy()
kat.lzm1_zm2.L = 2260e-3
kat.sOM1_OM2.L = 1.39
run_change_lens1(kat)

In [ ]:
def run_change_lens1_lens2(kat, dz1, dz2, f1, f2, plot=False, **kwargs):
    if plot:
        fig2 = plt.figure(figsize=(4*2, 4))
        ax2 = plt.subplot(111)

    kat.sqz_lens1.f = f1
    kat.sqz_lens2.f = f2
    
    OMCqx, OMCqy = cav2node('OMC', 'nSRM2', kat)
    OMCqx.reverse()
    OMCqy.reverse()

    trace_OMCx = kat.beamTrace(OMCqx, 'nSRM2', "nOMC_REFL",'x')
    trace_OMCy = kat.beamTrace(OMCqy, 'nSRM2', "nOMC_REFL",'y')

    if plot: trace_OMCy.plot_beamsize(ax=ax2, color='k', ls=':', label="OMC")
    if plot: trace_OMCx.plot_beamsize(ax=ax2, color='k', ls='--', label="OMC")

    kat.data['fn_OPO_shift_lenses'](kat, dz1, 0.5, dz2)
    
    trace_SQZ_SRMx = kat.beamTrace(kat.sqz.nSQZ.qx, 'nSQZ', "nSRMs2",'x')

    qx = single_bounce_SQZ(kat, 'x', False, 'nSRM2')

    trace_SRM_OMCx = kat.beamTrace(qx, 'nSRM2', "nOMC_REFL",'x')
    
    if plot: trace_SRM_OMCx.plot_beamsize(ax=ax2, **kwargs)

    trace_SQZ_SRMy = kat.beamTrace(kat.sqz.nSQZ.qy, 'nSQZ', "nSRMs2",'y')

    qy = single_bounce_SQZ(kat, 'y', False, 'nSRMs2')

    trace_SRM_OMCy = kat.beamTrace(qy, 'nSRMs2', "nOMC_REFL",'y')
    kwargs['ls'] = '--'
    if plot: trace_SRM_OMCy.plot_beamsize(ax=ax2, **kwargs)

    print("x mismatch", pykat.BeamParam.overlap(trace_SRM_OMCx.q_out, trace_OMCx.q_out))
    print("y mismatch", pykat.BeamParam.overlap(trace_SRM_OMCy.q_out, trace_OMCy.q_out))

    if plot:
        ax2.set_xlabel("SRM AR -> OMC REFL")
        ax2.legend()
#         ax2.set_ylim(0,3)

#         ax2.set_xlim(5.3, 6.5)
#         ax2.set_ylim(0, 1)

In [ ]:
kat = base.deepcopy()
kat.lzm1_zm2.L = 2260e-3
kat.sOM1_OM2.L = 1.39


run_change_lens1_lens2(kat, dz1, dz2, base.sqz_lens1.f.value*4, 0.75*2.2, plot=True, c='r')

In [ ]:
run_change_lens1_lens2(kat, dz1, dz2-2e-2, 0.250 *2.23, 0.350*2.23, plot=True, c='r')

In [ ]:
base.sqz_lens1.f.value, base.sqz_lens2.f.value